# 1. Thư viện

In [1]:
import json

from pyvi import ViTokenizer
from langchain_experimental.text_splitter import SemanticChunker
from langchain_community.embeddings import HuggingFaceEmbeddings

# 2. Model embedding

In [2]:
model_kwargs = {"device": "cuda"}
encode_kwargs = {"normalize_embeddings": True}
embeddings_Model = HuggingFaceEmbeddings(
    model_name="huyydangg/DEk21_hcmute_embedding",
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs
)

C:\Users\ADMIN\AppData\Local\Temp\ipykernel_48596\2870113787.py:3: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings_Model = HuggingFaceEmbeddings(


# 3. Khởi tạo Semantic Chunking

In [23]:
# Tạo SemanticChunker với các tham số phù hợp
chunker = SemanticChunker(
    embeddings = embeddings_Model, 
    buffer_size = 5,
    breakpoint_threshold_type = "gradient",
    breakpoint_threshold_amount = 0.75,
    sentence_split_regex = r"(?<=[.?!])\s+(?!\d+\.\s)",
    min_chunk_size = 512
)

# 4. Thực thi

In [24]:
# Đường dẫn file input và output
input_file_path = r"d:\13_21133021_NguyenTrongDung_LuuTruMonHoc\data_KLTN\trachNhiem_HinhSu\content_trachNhiem_HinhSu_Extracted.json"
output_file_path = r"d:\13_21133021_NguyenTrongDung_LuuTruMonHoc\data_KLTN\trachNhiem_HinhSu\semantic_Chunk_content_trachNhiem_HinhSu_Extracted.json"

In [25]:
# Đọc dữ liệu từ file .json
with open(input_file_path, 'r', encoding='utf-8') as file:
    data = json.load(file)

In [26]:
# Danh sách để lưu các đối tượng đã chunk
chunked_data = []

# Duyệt qua từng phần tử trong dữ liệu
for item in data:
    # Lấy nội dung của key "Semantic-Chunk-Content"
    content = item.get("SemanticChunk-Content", "")

    # Token hóa nội dung bằng ViTokenizer
    tokenized_content = ViTokenizer.tokenize(content)

    # Thực hiện chunking trên nội dung đã token hóa
    chunks = chunker.split_text(tokenized_content)

    # Lưu các chunk đã xử lý
    for chunk in chunks:
        chunk_data = item.copy()  # Tạo bản sao của metadata gốc
        chunk_data["SemanticChunk-Content"] = chunk  # Thay thế nội dung đã chunked
        chunked_data.append(chunk_data)

In [27]:
# Lưu kết quả vào file output (ví dụ: JSON)
with open(output_file_path, 'w', encoding='utf-8') as output_file:
    json.dump(chunked_data, output_file, ensure_ascii=False, indent=4)

print(f"File đã được lưu tại: {output_file_path}")

File đã được lưu tại: d:\13_21133021_NguyenTrongDung_LuuTruMonHoc\data_KLTN\trachNhiem_HinhSu\semantic_Chunk_content_trachNhiem_HinhSu_Extracted.json
